# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [16]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [17]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [18]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [19]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [20]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [21]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [22]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [23]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [24]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [25]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [26]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [27]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [28]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

2024-07-10 10:58:12,884 - 140669211241344 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [29]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [30]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is continuously monitoring and analyzing market data for AAPL in order to identify trends and predict market movements. I should delegate tasks and ask questions to gather the necessary insights and alerts about significant market opportunities or threats for AAPL.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL", "context": "Utilize statistical modeling and machine learning to identify trends and predict market movements for AAPL."}

> Entering new CrewAgentExecutor chain...
I need to start by gathering the most recent and relevant market data for AAPL in order to provide accurate insights.

Final Answer:
I recommend considering the following trading strategies based on the trends and predictions for AAPL:

1. Utilize the insights from TipRanks, which provides a 12-month price target forecast based on 35 analysts' predictions.
2. Monitor real-time forecasts and company news from CNN's stock quote for AAPL.
3. Analyze the stock price gains and trends from StockInvest.us to make informed decisions on when to buy or sell AAPL shares.
4. Consider long-term predictions for 2025 and 2030 from CoinCodex to align trading strategies with future price trends.
5. Evaluate analysts' opinions on future price estimates from TradingView, which includes a maximum estimate of $275.00 and a minimum estimate of $125.00 for AAPL.
6. Review the technical analysis and stock price forecast from Gov.Capital to gain insights into potential price movements.
7. Stay updated on the latest analyst stock forecasts, price targets, and recommendation trends from Yahoo Finance for AAPL.
8. Take into accou

Final Answer: The historical prices for Apple Inc. (AAPL) stock from July 9, 2023, to July 9, 2024, are as follows:

- July 9, 2024: Open - 227.93, High - 229.40, Low - 226.37, Close - 228.68, Volume - 48,076,100
- July 8, 2024: Open - 227.09, High - 227.85, Low - 223.25, Close - 227.82, Volume - 59,085,900
- July 5, 2024: Open - 221.65, High - 226.45, Low - 221.65, Close - 226.34, Volume - 60,412,400
- and so on...

This data provides a detailed overview of the historical prices for AAPL stock over the specified period, which can be further analyzed to gain insights into market trends and movements.

> Finished chain.
 

The historical prices for Apple Inc. (AAPL) stock from July 9, 2023, to July 9, 2024, are as follows:

- July 9, 2024: Open - 227.93, High - 229.40, Low - 226.37, Close - 228.68, Volume - 48,076,100
- July 8, 2024: Open - 227.09, High - 227.85, Low - 223.25, Close - 227.82, Volume - 59,085,900
- July 5, 2024: Open - 221.65, High - 226.45, Low - 221.65, Close - 226.34,

Final Answer: 
Based on the historical prices for AAPL stock from July 9, 2023, to July 9, 2024, obtained from Yahoo Finance, and considering the user-defined risk tolerance is set to Medium, I recommend the following potential trading strategies:

1. Trend Following Strategy: Utilize moving averages to identify the direction of the trend and make trades accordingly. When the stock price is above the moving average, consider buying, and when it's below, consider selling.

2. Mean Reversion Strategy: Look for situations where the stock price deviates significantly from its historical average and take trades to capitalize on the expected return to the mean.

3. Breakout Strategy: Monitor key support and resistance levels on the stock chart and execute trades when the price breaks out of these levels, indicating a potential strong movement in the direction of the breakout.

4. Momentum Strategy: Identify stocks that have shown strong positive momentum over the historical period and consid

Final Answer:
Title: Mastering covered calls: a strategic guide for Apple investors | Saxo
Link: https://www.home.saxo/content/articles/options/mastering-covered-calls---a-strategic-guide-for-apple-investors-03062024
Snippet: This article provides a comprehensive guide to using covered calls for generating additional income with Apple (AAPL) shares.

Title: Stock Trading in AAPL: 27 Things Traders Should Know About
Link: https://www.quantifiedstrategies.com/stock-trading-aapl/
Snippet: AAPL stocks can be purchased through a broker, and the best trading strategy depends on individual preferences, risk tolerance, and understanding of the market.

Title: Apple Stock Trading | Great conditions - AvaTrade
Link: https://www.avatrade.com/trading-info/financial-instruments-index/stocks/apple
Snippet: Apple stock trades on the NASDAQ exchange under the Global Select Market category with the ticker symbol AAPL. Apple has been renowned as one of the most popular stocks to trade.

Title: AAPL Bull

Thought: I need to gather detailed information on the proposed trading strategies for AAPL and assess the associated risks to suggest mitigation strategies.

Action: Read website content
Action Input: {"website_url": "https://www.fxleaders.com/stock-signals/how-to-trade-aapl-apple-stock/"} 

How to Trade (AAPL) Apple Stock - FX Leaders Stock Signals
 FREE SIGNALS
<% signal.pair %> <% signal.rate %>
|
OPEN A FUTURE TRADING ACCOUNT WITH PLUS500
Italiano 
Deutsch 
عَرَبِيّ Sign in
News
Forex
Bitcoin
Gold
EUR/USD
Africa News
Analysis
Asia News
Commodities News
Cryptocurrency News
Currencies News
Economic Indicators
Europe
Latam
Stock Markets
Opinions
USA
Live Rates
Forex rates
EUR/USD
USD/JPY
GBP/USD
USD/CHF
USD/CAD
AUD/USD
NZD/USD
Crypto rates
BITCOIN
ETHEREUM
SOLANA
RIPPLE
DOGE
BNB
CARDANO
AVALANCHE
Stocks rates
TESLA
APPLE
COINBASE
GOOGLE
FACEBOOK
AMAZON
NETFLIX
PFIZER
Indices rates
DOW
S&P500
DAX
FTSE
NIKKEI225
NAS100
Commodities rates
GOLD
SILVER
USOIL
UKOIL
PLATINUM
PALLADIUM
Forex S

Final Answer: 

After conducting a search on the internet, I found an article on FX Leaders that provides detailed information on trading strategies for Apple (AAPL) stock. The article covers various aspects, including understanding Apple's financials, technical analysis, fundamental analysis, different trading strategies (such as day trading, swing trading, and long-term investing), and tips for managing risk and maximizing returns.

The proposed trading strategies for AAPL mentioned in the article include day trading, swing trading, and long-term investing. Day trading involves buying and selling the stock within a single trading day, while swing trading involves holding the stock for a few days to a couple of weeks. Long-term investing, on the other hand, involves buying the stock with the intention of holding it for an extended period.

To mitigate risks associated with these strategies, it is essential to diversify the portfolio, set stop-loss orders, and have a well-defined inves

- Display the final result as Markdown.

In [31]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report:

After evaluating the proposed trading strategies for AAPL mentioned in the articles, it is evident that there are several potential risks associated with each strategy. 

1. Day Trading: The main risk with day trading AAPL stock is the high volatility and rapid price movements that can lead to substantial losses if not managed properly. Additionally, the need for constant monitoring of the market and making quick decisions can result in emotional trading and impulsive decision-making.

Mitigation Strategy: To mitigate the risks of day trading, it is essential to set strict stop-loss orders, adhere to a disciplined trading plan, and avoid overtrading. Implementing risk management techniques such as position sizing and diversification can also help in minimizing potential losses.

2. Swing Trading: While swing trading AAPL stock allows for more flexibility in holding periods compared to day trading, it still carries risks such as overnight gaps in price and unexpected news events that can impact the stock price negatively. Moreover, swing trading requires a thorough understanding of technical analysis and market trends to make informed trading decisions.

Mitigation Strategy: To mitigate the risks of swing trading, it is crucial to conduct in-depth technical analysis, identify key support and resistance levels, and use trailing stop orders to protect profits. Additionally, staying updated on company news and market developments can help in anticipating potential price movements.

3. Long-Term Investing: Although long-term investing in AAPL stock is generally considered less risky compared to day trading and swing trading, it is not without risks. Factors such as market downturns, economic uncertainties, and company-specific risks can impact the long-term performance of the stock.

Mitigation Strategy: To mitigate the risks of long-term investing, it is important to diversify the investment portfolio, conduct thorough fundamental analysis of the company, and stay informed about macroeconomic trends that could affect the stock price. Setting realistic investment goals and having a long-term perspective can help in navigating market fluctuations and achieving sustainable returns.

In conclusion, while the proposed trading strategies for AAPL offer opportunities for generating additional income and optimizing trading approaches, it is crucial to be aware of the potential risks involved and implement appropriate risk mitigation strategies to safeguard investment capital. By diversifying trading strategies, setting clear risk management rules, and staying informed about market conditions, traders can enhance their chances of success in trading AAPL stock.